Loading the csv file into a dataframe to read it


In [ ]:
df = spark.read.format("csv").load("Files/bronze/covid_data.csv")
display(df)

printing the schema 

In [ ]:
df.printSchema()

saving bronze dataset as a parquet file

In [ ]:
new_df.write.mode("overwrite").parquet("Files/Bronze/BronzeDataset.parquet")

Saving the bronze dataset as a table

In [ ]:
new_df.write.mode("overwrite").saveAsTable("BronzeTable")

creating our silver layer dataframe

In [ ]:
silver_df = spark.read.parquet("Files/Bronze/BronzeDataset.parquet")
display(silver_df)

Casting Columns in our silver layer dataframe

In [ ]:
from pyspark.sql.types import IntegerType

silver_df = silver_df.withColumn("Confirmed", col("Confirmed").cast(IntegerType()))
silver_df = silver_df.withColumn("Deaths", col("Deaths").cast(IntegerType()))
silver_df = silver_df.withColumn("Recovered", col("Recovered").cast(IntegerType()))

silver_df.printSchema()

storing silver dataframe as a parquet file

In [ ]:
silver_df.write.mode("overwrite").parquet("Files/Silver/SilverDataset.parquet")

storing as table

In [ ]:
silver_df.write.mode("overwrite").saveAsTable("SilverTable")